# Data Visualization with GeoPy, Folium and hvplot

**GeoPy**: a Python client to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources. https://geopy.readthedocs.io/en/stable/
 
**Folium**: to visualize data that’s been manipulated in Python on an interactive leaflet map. http://python-visualization.github.io/folium/

**hvPlot**: a high-level plotting API built on HoloViews that provides a general and consistent API to create interactively explorable Bokeh plots with panning, zooming, hovering, and clickable/selectable legends. https://hvplot.holoviz.org/

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import intake
from ipywidgets import widgets
from geopy.geocoders import Nominatim
import folium 
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore")
from os.path import expanduser
home = expanduser("~")

### Choose a location
<a id='selection'></a>

In [ ]:
place_box = widgets.Text(description="Enter place:", value="Taipei")
display(place_box)

In [ ]:
# The Nominatim module is used to retrieve the geographical coordinates of the selected place

geolocator = Nominatim(user_agent="any_agent")
location = geolocator.geocode(place_box.value)

print(location.address)
print((location.latitude, location.longitude))

In [ ]:
# The folium package is used to plot our selected geolocation on a map

m = folium.Map(location=[location.latitude, location.longitude])
tooltip = location.latitude, location.longitude
folium.Marker([location.latitude, location.longitude], tooltip=tooltip).add_to(m)
display(m)

### Select a dataset

In [ ]:
esm_file = home+"/CMIP6_ESM_collection_file_datahub.json"
col = intake.open_esm_datastore(esm_file)

In [ ]:
query = dict(source_id="CMCC-ESM2",
             experiment_id="ssp585",
             variable_id="tasmax",
             table_id="day"
)
cat = col.search(**query)
cat.df

### Choose a time interval

In [ ]:
start = widgets.SelectionSlider(
    options=range(int(min(cat.df["start_year"])),int(max(cat.df["end_year"]))+1),
    value=int(min(cat.df["start_year"])),
    description='Start Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
display(start)

In [ ]:
end = widgets.SelectionSlider(
    options=range(start.value,int(max(cat.df["end_year"]))+1),
    value=start.value,
    description='End Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
display(end)

### Open multiple files as a single dataset and subset over the chosen time interval

In [ ]:
dset_dict = xr.open_mfdataset(cat.df["path"], parallel=True,engine="netcdf4")
dset_dict

In [ ]:
tas_year_xr = dset_dict["tasmax"].sel(time=dset_dict.time.dt.year.isin(range(start.value, end.value+1)))
tas_year_xr

### Find the nearest model coordinate

by finding the index of the nearest grid point

In [ ]:
def find_nearest(lat_input, long_input):

    lat_index  = np.nanargmin((tas_year_xr["lat"]-lat_input)**2)
    lon_index = np.nanargmin((tas_year_xr["lon"]-long_input)**2)
    return lon_index,lat_index

([xloc, yloc]) = find_nearest(location.latitude,location.longitude) # xloc and yloc are the indices of the nearest model grid point

### Draw the map again

In [ ]:
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=8)

tooltip = location.latitude, location.longitude
folium.Marker(
    [location.latitude, location.longitude],
    tooltip=tooltip,
    popup="Location selected by You",
).add_to(m)


tooltip = float(tas_year_xr["lat"][yloc].values), float(tas_year_xr["lon"][xloc].values)
folium.Marker(
    [tas_year_xr["lat"][yloc], tas_year_xr["lon"][xloc]],
    tooltip=tooltip,
    popup="Model Grid Cell Center",
).add_to(m)


# Define coordinates of model grid cell (just for visualization)

rect_lat1_model = (tas_year_xr["lat"][yloc - 1] + tas_year_xr["lat"][yloc]) / 2
rect_lon1_model = (tas_year_xr["lon"][xloc - 1] + tas_year_xr["lon"][xloc]) / 2
rect_lat2_model = (tas_year_xr["lat"][yloc + 1] + tas_year_xr["lat"][yloc]) / 2
rect_lon2_model = (tas_year_xr["lon"][xloc + 1] + tas_year_xr["lon"][xloc]) / 2


# Draw model grid cell

folium.Rectangle(
    bounds=[[rect_lat1_model, rect_lon1_model], [rect_lat2_model, rect_lon2_model]],
    color="#ff7800",
    fill=True,
    fill_color="#ffff00",
    fill_opacity=0.2,
).add_to(m)

m

### Get data for the nearest model coordinate and plot it on a interactive graph
 - Select the chosen spatial point
 - Convert Kelvin to °C
 - Convert `CFTimeIndex` to a Pandas `DatetimeIndex` to avoid plotting issues (labels, indexes, ...) 
 - Create a dataframe to accommodate a) the selected model data and b) the Summer Day Threshold
 - Plot data with `hvplot`

In [ ]:
%%time
tasmax_year_place_xr = tas_year_xr[:, yloc, xloc] - 273.15
tasmax_year_place_xr["time"] = tasmax_year_place_xr.indexes["time"].to_datetimeindex()
tasmax_year_place_xr

In [ ]:
%%time
tasmax_year_place_df = pd.DataFrame(index = tasmax_year_place_xr['time'].values, 
                                    columns = ['Model Temperature', 'Summer Day Threshold'])

tasmax_year_place_df.loc[:, 'Model Temperature'] = tasmax_year_place_xr.values # Model data
tasmax_year_place_df.loc[:, 'Summer Day Threshold'] = 25                       # Summer Day Threshold

# Plot data and define title and legend
tasmax_year_place_df.hvplot.line(y=['Model Temperature', 'Summer Day Threshold'], 
                                 value_label='Temperature in °C', legend='bottom', 
                                 title='Daily-Maximum Near-Surface Air Temperature for '+place_box.value, 
                                 height=500, width=620)

### Compute the Summer days climate index

Starting from the daily maximum temperature `TX`,
the **Summer Days** index is the number of days where $TX > T$ (`T` is  a reference temperature, e.g. 25°C)

In [ ]:
%%time
summer_days = xr.where(tasmax_year_place_xr>25.0, 1, 0).groupby('time.year').sum()
summer_days

In [ ]:
%%time
summer_days_df = pd.DataFrame(index = summer_days['year'].values, 
            columns = ['Summer Days'])

summer_days_df.loc[:, 'Summer Days'] = summer_days.values

summer_days_df.hvplot.line(y=['Summer Days'],
            value_label='Summer Days', legend='bottom', 
            title='Number of Summer Days in '+place_box.value+" in "+str(start.value)+"-"+str(end.value), 
            height=500, width=620)